In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import models
from torch.utils.data import DataLoader, random_split, Dataset
from PIL import Image
import glob
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda")


In [ ]:
class PlantDiseaseDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        
        for class_name in self.classes:
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                for img_path in glob.glob(f"{class_path}/*.{ext}") for ext in ['jpg', 'png', 'jpeg']:
                    self.image_paths.append(img_path)
                    self.labels.append(self.class_to_idx[class_name])
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image.to(device), torch.tensor(label, dtype=torch.long, device=device) 

In [ ]:
root_dir = r"C:\Users\Batuhan\Desktop\VeriSetOrj224" 

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = PlantDiseaseDataset(root_dir, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)


In [ ]:
model = models.resnet50(pretrained=True).to(device)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(dataset.classes)).to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)


In [ ]:
early_stopping_patience = 3
best_val_loss = float('inf')
patience_counter = 0

total_epochs = 20
for epoch in range(total_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_train_loss = running_loss / len(train_loader)
    avg_val_loss = val_loss / len(val_loader)
    accuracy = 100 * correct / total

    print(f"Epoch {epoch+1}/{total_epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%")

    scheduler.step()

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0  
        torch.save(model.state_dict(), "best_model.pth")  
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print("Erken durdurma etkin! Eğitim durduruldu.")
            break

In [ ]:
torch.save(model.state_dict(), "final_plant_disease_model.pth")

In [ ]:
unknown_threshold = 0.5 

def predict_with_unknown(model, image_path, transform, device, threshold=unknown_threshold):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output, dim=1)  
        max_prob, predicted_class_idx = torch.max(probabilities, 1) 
        
    if max_prob.item() < threshold:
        predicted_class = "unknown"
    else:
        predicted_class = dataset.classes[predicted_class_idx.item()]

    return predicted_class, max_prob.item()

image_path = "path_to_test_image.jpg"
predicted_class, confidence = predict_with_unknown(model, image_path, transform, device)

print(f'Tahmin Edilen Sınıf: {predicted_class} (Güven: {confidence:.2f})')
